## Importing but crashing

In [ ]:
import os
from copy import deepcopy

import numpy as np

import pydicom
from pymedphys.deliverydata import DeliveryData
from pymedphys.dcm import anonymise_dicom
from pymedphys.coll import find_relevant_control_points

In [ ]:
# from decode_trf import *

In [ ]:
from pymedphys.logfile import delivery_data_from_logfile

In [ ]:
logfile_path = os.path.join(
    '..', '..',
    'tests', 'data', 'trf', 'integrity4',
    'INT4 18_12_12 08_28_23 Z [ICOM_ 7_03_2018 15_21_51] PF_G90..trf'
)
logfile_path

In [ ]:
delivery_data = delivery_data_from_logfile(logfile_path)

In [ ]:
# relevant_cp = find_relevant_control_points(delivery_data.monitor_units)

# remove_irrelevant_control_points(*delivery_data)

In [ ]:
def filter_out_irrelivant_control_points(delivery_data):
    
    relvant_control_points = find_relevant_control_points(
        delivery_data.monitor_units)
    
    new_delivery_data = []
    for item in delivery_data:
        new_delivery_data.append(np.array(item)[relvant_control_points].tolist())
        
    return DeliveryData(*new_delivery_data)


delivery_data = filter_out_irrelivant_control_points(delivery_data)



In [ ]:
# delivery_data

In [ ]:
np.shape(np.array(delivery_data.mlc))


mlc = np.array(delivery_data.mlc)

np.shape(mlc[:,:,0])

In [ ]:
def convert_mlc_format(mlc):
    
    dicom_mlc_format = []
    for control_point in mlc:
        concatenated = np.hstack([-control_point[:,0], control_point[:,1]])
        dicom_mlc_format.append(concatenated.astype(str).tolist())
        
    return dicom_mlc_format
        
converted_mlc = convert_mlc_format(mlc)

In [ ]:
# delivery_data.jaw

In [ ]:
# converted_mlc

In [ ]:
# delivery_data.jaw

In [ ]:
# for item in mlc:
#     print(np.shape(item))

In [ ]:
filepath = '../../tests/data/dcmplan/TinyVMAT/TinyFS_TinyVMAT.dcm'
filepath_out = '../../tests/data/dcmplan/test/test.dcm'
dcm = pydicom.read_file(filepath, force=True)

In [ ]:
# dcm = anonymise_dicom(dcm)

In [ ]:
# dcm

In [ ]:
# dcm

In [ ]:
# dcm

In [ ]:
control_point_sequence = dcm.BeamSequence[0].ControlPointSequence

In [ ]:
# control_point_sequence[0]

In [ ]:
# [
#     controlpoint.GantryAngle
#     for controlpoint in control_point_sequence
# ]

In [ ]:
# converted_mlc[0]

In [ ]:
dummy_mlc_pos_neg = [-1.8,]*30 + [-100,]*20 + [-1.8,]*30
dummy_mlc_pos_floats = dummy_mlc_pos_neg + (-1 * np.array(dummy_mlc_pos_neg)).tolist()
dummy_mlc_pos_string = np.array(dummy_mlc_pos_floats).astype(str).tolist()

In [ ]:
len(delivery_data.monitor_units)

In [ ]:
skip_step = 24

monitor_units_trim = np.concatenate(
    [np.array(delivery_data.monitor_units)[::skip_step],
     np.array(delivery_data.monitor_units)[-1::]]
).tolist()
converted_mlc_trim = np.concatenate([
    converted_mlc[::skip_step], converted_mlc[-1::]
]).tolist()


# monitor_units_trim

In [ ]:
monitor_units_trim[-1]

In [ ]:
delivery_data.monitor_units[-1]

In [ ]:
# init_cp = deepcopy(control_point_sequence[0])
# subsequent_cp = deepcopy(control_point_sequence[-1])

# init_cp.GantryAngle = "90.0"
# init_cp.GantryRotationDirection = "NONE"
# init_cp.BeamLimitingDevicePositionSequence[0].LeafJawPositions = ['-100.0', '100.0']

# # init_cp.BeamLimitingDevicePositionSequence[1].LeafJawPositions = converted_mlc_trim[0]
# init_cp.BeamLimitingDevicePositionSequence[1].LeafJawPositions = dummy_mlc_pos_string

# subsequent_cp.GantryAngle = "90.0"
# subsequent_cp.BeamLimitingDevicePositionSequence[0].LeafJawPositions = ['-100.0', '100.0']

# current_cp = deepcopy(subsequent_cp)
# current_cp.ControlPointIndex = "1"
# #     current_cp.BeamLimitingDevicePositionSequence[1].LeafJawPositions = mlc_cp
# current_cp.BeamLimitingDevicePositionSequence[1].LeafJawPositions = dummy_mlc_pos_string
# current_cp.CumulativeMetersetWeight = "1.000000"


# all_control_points = [init_cp] + [current_cp]

In [ ]:
# converted_mlc_trim[0]

In [ ]:
init_cp = deepcopy(control_point_sequence[0])
subsequent_cp = deepcopy(control_point_sequence[-1])

init_cp.GantryAngle = "90.0"
init_cp.GantryRotationDirection = "NONE"
init_cp.BeamLimitingDevicePositionSequence[0].LeafJawPositions = ['-130.0', '130.0']

init_cp.BeamLimitingDevicePositionSequence[1].LeafJawPositions = converted_mlc_trim[0]
# init_cp.BeamLimitingDevicePositionSequence[1].LeafJawPositions = dummy_mlc_pos_string

subsequent_cp.GantryAngle = "90.0"
subsequent_cp.BeamLimitingDevicePositionSequence[0].LeafJawPositions = ['-130.0', '130.0']

remaining_cps = []
for i, (mu, mlc_cp) in enumerate(zip(monitor_units_trim[1::], converted_mlc_trim[1::])):    
    current_cp = deepcopy(subsequent_cp)
    current_cp.ControlPointIndex = str(i+1)
    current_cp.BeamLimitingDevicePositionSequence[1].LeafJawPositions = mlc_cp
    current_cp.CumulativeMetersetWeight = np.around(
        mu / delivery_data.monitor_units[-1], decimals=5)
    
    remaining_cps.append(current_cp)
    

all_control_points = [init_cp] + remaining_cps

In [ ]:
# dcm = pydicom.read_file(filepath, force=True)
# dcm.save_as(filepath_out)

In [ ]:
# edited_dcm = deepcopy(dcm)

In [ ]:
# dcm = pydicom.read_file(filepath, force=True)

# edited_dcm = deepcopy(dcm)

# # first_cp = edited_dcm.BeamSequence[0].ControlPointSequence[0]
# # last_cp = edited_dcm.BeamSequence[0].ControlPointSequence[-1]

# # last_cp.ControlPointIndex = "1"
# # last_cp.GantryAngle = "200.0"


# # # edited_dcm.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset = (
# # #     str(monitor_units_trim[-1]))


# # edited_dcm.BeamSequence[0].ControlPointSequence = [
# #     first_cp, last_cp
# # ]
# # edited_dcm.BeamSequence[0].NumberOfControlPoints = "2"

# edited_dcm.save_as(filepath_out)
# edited_dcm

In [ ]:
# remaining_cps

In [ ]:
edited_dcm = deepcopy(dcm)


edited_dcm.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset = (
    str(monitor_units_trim[-1]))


edited_dcm.BeamSequence[0].ControlPointSequence = all_control_points
edited_dcm.BeamSequence[0].NumberOfControlPoints = len(all_control_points)

edited_dcm.save_as(filepath_out)

In [ ]:
edited_dcm

In [ ]:
DataElement = pydicom.dataelem.DataElement
Dataset = pydicom.dataset.Dataset

In [ ]:
dummy_mlc_pos_neg = [-1.8,]*30 + [-100,]*20 + [-1.8,]*30
dummy_mlc_pos_floats = dummy_mlc_pos_neg + (-1 * np.array(dummy_mlc_pos_neg)).tolist()
dummy_mlc_pos_string = np.array(dummy_mlc_pos).astype(str).tolist()

In [ ]:
def create_control_point(index, gantry_angle, gantry_rotation, mu):

    ds = Dataset()
    ds.ControlPointIndex = str(index)

    ds.NominalBeamEnergy = '6.0'

    asymy = Dataset()
    asymy.RTBeamLimitingDeviceType = 'ASYMY'
    asymy.LeafJawPositions = ['-100.0', '100.0']

    mlcx = Dataset()
    mlcx.RTBeamLimitingDeviceType = 'MLCX'
    mlcx.LeafJawPositions = dummy_mlc_pos_string

    ds.BeamLimitingDevicePositionSequence = [asymy, mlcx]

    ds.GantryAngle = str(float(gantry_angle))
    ds.GantryRotationDirection = gantry_rotation

    ds.BeamLimitingDeviceAngle = '0.0'
    ds.BeamLimitingDeviceRotationDirection = 'NONE'

    ds.CumulativeMetersetWeight = str(float(mu))

    return ds


beam_sequence = Dataset()
beam_sequence.BeamNumber = "1"
beam_sequence.NumberOfControlPoints = "2"
beam_sequence.ControlPointSequence = [
    create_control_point(0, 270, 'CW', 0),
    create_control_point(1, 90, 'NONE', 1)
]

beam_sequence


edited_dcm.BeamSequence = [beam_sequence]
edited_dcm

edited_dcm.save_as(filepath_out)

In [ ]:
# pydicom.uid.generate_uid()

In [ ]:
# pydicom.dataset.Dataset({
#     (0x300a, 0x0112): DataElement(0x300a0112, 'IS', '1')
# })

In [ ]:
pydicom.dataset.Dataset?